# Exercise 9 - Using the GPU API

**GOAL:** The goal of this exercise is to show how to use GPUs with remote functions and actors.

**NOTE:** These exercises are designed to run on a machine without GPUs.

See the documentation on using Ray with GPUs http://ray.readthedocs.io/en/latest/using-ray-with-gpus.html.

### Concepts for this Exercise - Using Ray with GPUs

We can indicate that a remote function or an actor requires some GPUs using the `num_gpus` keyword.

```python
@ray.remote(num_gpus=1)
def f():
    # The command ray.get_gpu_ids() returns a list of the indices
    # of the GPUs that this task can use (e.g., [0] or [1]).
    ray.get_gpu_ids()

@ray.remote(num_gpus=2)
class Foo(object):
    def __init__(self):
        # The command ray.get_gpu_ids() returns a list of the
        # indices of the GPUs that this actor can use
        # (e.g., [0, 1] or [3, 5]).
        ray.get_gpu_ids()
```

Then inside of the actor constructor and methods, we can get the IDs of the GPUs allocated for that actor with `ray.get_gpu_ids()`.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import ray
import time

Start Ray, note that we pass in `num_gpus=4`. Ray will assume this machine has 4 GPUs (even if it does not). When a task or actor requests a GPU, it will be assigned a GPU ID from the set `[0, 1, 2, 3]`. It is then the responsibility of the task or actor to make sure that it only uses that specific GPU (e.g., by setting the `CUDA_VISIBLE_DEVICES` environment variable).

In [2]:
ray.init(num_cpus=4, num_gpus=2, include_webui=False, ignore_reinit_error=True)

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-01-24_10-10-42_21653/logs.
Waiting for redis server at 127.0.0.1:62086 to respond...
Waiting for redis server at 127.0.0.1:46207 to respond...
Starting the Plasma object store with 20.0 GB memory using /dev/shm.


{'node_ip_address': '192.168.23.45',
 'redis_address': '192.168.23.45:62086',
 'object_store_addresses': ['/tmp/ray/session_2019-01-24_10-10-42_21653/sockets/plasma_store'],
 'raylet_socket_names': ['/tmp/ray/session_2019-01-24_10-10-42_21653/sockets/raylet'],
 'webui_url': ''}

**EXERCISE:** Change the remote function below to require one GPU.

**NOTE:** This change does not make the remote function actually **use** the GPU, it simply **reserves** the GPU for use by the remote function. To actually use the GPU, the remote function would use a neural net library like TensorFlow or PyTorch after setting the `CUDA_VISIBLE_DEVICES` environment variable properly. This can be done as follows.

```python
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ','.join([str(i) for i in ray.get_gpu_ids()])
```

In [7]:
@ray.remote(num_gpus=1)
def f():
    time.sleep(0.5)
    return ray.get_gpu_ids()

**VERIFY:** This code checks that each task was assigned one GPU and that not more than two tasks are run at the same time (because we told Ray there are only two GPUs).

In [8]:
start_time = time.time()

gpu_ids = ray.get([f.remote() for _ in range(3)])

end_time = time.time()

for i in range(len(gpu_ids)):
    assert len(gpu_ids[i]) == 1

assert end_time - start_time > 1

print('Sucess! The test passed.')

Sucess! The test passed.


**EXERCISE:** The code below defines an actor. Make it require one GPU.

In [5]:
@ray.remote(num_gpus=1)
class Actor(object):
    def __init__(self):
        pass

    def get_gpu_ids(self):
        return ray.get_gpu_ids()

**VERIFY:** This code checks that the actor was assigned a GPU.

In [6]:
actor = Actor.remote()

gpu_ids = ray.get(actor.get_gpu_ids.remote())

assert len(gpu_ids) == 1

print('Sucess! The test passed.')

AssertionError: 